In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV





data = pd.read_csv("data/variable_interval_aggregate_train.csv", skip_blank_lines=False, index_col=0, infer_datetime_format=True)

